# Data Import & Cleaning

In [ ]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing as pp
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import make_column_transformer
from scipy.stats.mstats import winsorize

## Data Import 

In [ ]:
## Read data
folder = os.path.join(os.getcwd(), "Data/")
files = os.listdir(folder)

data_part1 = pd.read_csv(folder+files[0], delimiter='\t')
data_part2 = pd.read_csv(folder+files[1], delimiter='\t')
data_part3 = pd.read_csv(folder+files[2], delimiter='\t')
data_part4 = pd.read_csv(folder+files[3], delimiter='\t')

data_list = [data_part1, data_part2, data_part3, data_part4]
data = pd.concat(data_list)
data = data.reset_index(drop=True)
data.replace('nan',np.NaN)

#data.iloc[109161,:]
data


In [ ]:
## Handling wrong data

# Dropping distorted row (see below)
#data = data.drop(208166) # -> make this less static! (TBD) -> Collab
data = data.drop(109161) # -> make this less static! (TBD) -> VSCode

# Set type of numeric columns that weren't numeric
data["revol_util"] = pd.to_numeric(data["revol_util"])
data["total_acc"] = pd.to_numeric(data["total_acc"])
data["mort_acc"] = pd.to_numeric(data["mort_acc"])

# Get states for address
data['address'] = data['address'].str[-8:-6]

# Get only year for issue_d and earliest_cr_line
data['issue_d'] = data['issue_d'].str[4:]
data['earliest_cr_line'] = data['earliest_cr_line'].str[4:]

# settings to display all columns
pd.set_option("display.max_columns", None)

# show data
data
#data.sort_values('open_acc', ascending=False).head()

Handling of data:
*   Index 109161 (VSC) [or 208166 in Collab] distorted data bcs field "title" included also "dti" and others (mistake in .txt file) -> removed from dataset
*   3 columns that should be numeric aren't. Hence datatype made numeric:
    *   revol_util
    *   total_acc
    *   mort_acc
*   annual_inc: 1x 0 -> dti 9999 -> how to handle? -> falls probably into outlieer

## Data Cleaning



### Summary Statistics (before)

In [ ]:
# Data length and info
n_data = len(data)
print("Number of data points: "+str(n_data)+"\n")

data.info()


In [ ]:
# Splitting numerical and categorical variables
data_num = data.select_dtypes(include=['float64','int64'])
data_cat = data.select_dtypes(include=['object'])

In [ ]:
# Summary for categorical data
data_cat.describe().T

In [ ]:
# Summary for numerical data
pd.set_option('display.float_format', lambda x: '%.2f' % x)
data_num.describe()


Possible Outliers:
*   dti -> income=0, credit_card, charged off



In [ ]:
## Count NaN
# compute number of NaN values and percentage of NaN values for each column
nan_count = data.isna().sum()
nan_pct = data.isna().sum()/n_data

# create a new dataframe to store the results
nans = pd.DataFrame({'nan_count': nan_count, 'nan_pct': nan_pct})
nans["nan_pct_str"] = pd.Series(["{0:.2f}%".format(val * 100) for val in nans["nan_pct"]], index = nans.index)
nans

In [ ]:
# Value count for categorical data
vcs = dict()

for col in data_cat.columns:
    vc = data[col].value_counts().to_frame()
    vc["pct"] =  vc[col] / vc[col].sum()
    vc["pct"] = pd.Series(["{0:.2f}%".format(val * 100) for val in vc["pct"]], index = vc.index)
    vcs["vc {0}".format(col)] = vc
    display(vc)

**Target Variable**

In [ ]:
# Change target variable to dummy
data['loan_status'] = data['loan_status'].map({'Charged Off': 1, 'Fully Paid': 0})
data

In [ ]:
# Value count loan_status
vc_loanstatus = data["loan_status"].value_counts().to_frame()
vc_loanstatus["pct"] =  vc_loanstatus["loan_status"] / vc_loanstatus["loan_status"].sum()
vc_loanstatus["pct"] = pd.Series(["{0:.2f}%".format(val * 100) for val in vc_loanstatus["pct"]], index = vc_loanstatus.index)
display(vc_loanstatus)

# Plotting loan_status
sns.countplot(x=data["loan_status"], palette = "Set2")
plt.xticks(rotation=0)
plt.show()

# Correlation of variables with loan_status
cor_target = data.corrwith(data["loan_status"])
cor_target.sort_values(axis = 0, ascending = False)

### Useless & New Variables
*   Paper: If train/test split 70/30 -> don’t use variables having >= 30% missing values -> we could do that 
*   Dropping useless variables: 
    *  title
    * emp_title (maybe?) -> annual_inc might be correlated and more important
    * earliest_cr_line
    * address
* Add/Create variables?
    * Macroeconomic information?




In [ ]:
# Dropping useless variables
data_cleand = data.drop(['title','emp_title', 'earliest_cr_line', 'address'], axis=1)

# Dropping variables having more than train_ratio% of nan values
train_ratio = 0.7
test_ratio = 0.3
data_cleand = data_cleand.loc[:, nans["nan_pct"] <= test_ratio]
data_cleand


In [ ]:
# Splitting numerical and categorical variables
data_num_clean = data_cleand.select_dtypes(include=['float64','int64'])
data_cat_clean = data_cleand.select_dtypes(include=['object'])

### Data Visualizations 
* Distribution
* Boxplot

To visualize outliers and show in presentation
-> then outliers removed or winsorized


In [ ]:
## Plotting Distribution for non-numeric data (only those that don't have many categories)

#dist_nn = ["term", "emp_length", "home_ownership", "verification_status", "loan_status", "purpose", "initial_list_status", "application_type", "issue_d", "earliest_cr_line", "address"]
dist_nn = ["term", "emp_length", "home_ownership", "verification_status", "loan_status", "purpose", "initial_list_status", "application_type", "issue_d"]

fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(15, 15), sharex = False, sharey = False)
axes = axes.ravel()  

for col, ax in zip(dist_nn, axes): 
    # absolute
    sns.countplot(data=data_cleand, x=col, palette = "Set2", ax=ax)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

fig.delaxes(axes[11])
fig.tight_layout()
plt.show()

In [ ]:
## Plotting Distribution for non-numeric data (only those that don't have many categories)

fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(15, 15), sharex = False, sharey = False)
axes = axes.ravel()  

for col, ax in zip(dist_nn, axes): 
    # default vs non-default
    sns.countplot(data=data_cleand, x=col, hue="loan_status", palette='Set2', ax=ax)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

fig.delaxes(axes[11])    
fig.tight_layout()
plt.show()



In [ ]:
## Plotting Distribution for numeric data

fig, axs = plt.subplots(nrows=3, ncols=4, figsize=(15, 7))

# loan_amnt -> right/positive skewed
sns.histplot(data=data_cleand, x="loan_amnt", binwidth=2000, color="darkblue", kde=True, kde_kws={'bw_method': 0.4}, ax=axs[0,0])
# int_rate -> right/positive skewed
sns.histplot(data=data_cleand, x="int_rate", binwidth=1, color="darkblue", kde=True, kde_kws={'bw_method': 0.4}, ax=axs[0,1])
# installment -> right/positive skewed
sns.histplot(data=data_cleand, x="installment", binwidth=50, color="darkblue", kde=True, kde_kws={'bw_method': 0.4}, ax=axs[0,2])
# annual_inc -> right/positive skewed -> outlier -> re-do without
sns.histplot(data=data_cleand, x="annual_inc", binwidth=10000, color="darkblue", kde=True, kde_kws={'bw_method': 0.4}, ax=axs[0,3])
# dti -> right/positive skewed -> outlier -> re-do without
sns.histplot(data=data_cleand, x="dti", binwidth=1000, color="darkblue", kde=True, kde_kws={'bw_method': 0.4}, ax=axs[1,0])
# open_acc -> right/positive skewed
sns.histplot(data=data_cleand, x="open_acc", binwidth=2, color="darkblue", kde=True, kde_kws={'bw_method': 0.4}, ax=axs[1,1])
# pub_rec -> right/positive skewed -> outlier -> re-do without
sns.histplot(data=data_cleand, x="pub_rec", binwidth=1, color="darkblue", kde=True, kde_kws={'bw_method': 0.4}, ax=axs[1,2])
# revol_bal -> right/positive skewed -> outlier -> re-do without
sns.histplot(data=data_cleand, x="revol_bal", binwidth=10000, color="darkblue", kde=True, kde_kws={'bw_method': 0.4}, ax=axs[1,3])
# revol_util -> right/positive skewed -> outlier -> re-do without
sns.histplot(data=data_cleand, x="revol_util", binwidth=20, color="darkblue", kde=True, kde_kws={'bw_method': 0.4}, ax=axs[2,0])
# total_acc -> right/positive skewed -> outlier -> re-do without
sns.histplot(data=data_cleand, x="total_acc", binwidth=5, color="darkblue", kde=True, kde_kws={'bw_method': 0.4}, ax=axs[2,1])
# mort_acc -> right/positive skewed -> outlier -> re-do without
sns.histplot(data=data_cleand, x="mort_acc", binwidth=1, color="darkblue", kde=True, kde_kws={'bw_method': 0.4}, ax=axs[2,2])
# pub_rec_bankruptcies
sns.histplot(data=data_cleand, x="pub_rec_bankruptcies", binwidth=1, color="darkblue", kde=True, kde_kws={'bw_method': 0.4}, ax=axs[2,3])

fig.tight_layout()
plt.show()



In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(15, 7), sharex = False, sharey = False)
axes = axes.ravel()  
cols = data_num_clean.columns[:]

for col, ax in zip(cols, axes):
    d = data_num_clean 
    sns.kdeplot(data=d, x=col, shade=True, ax=ax)
    ax.set(title=f'Distribution of Variable: {col}', xlabel=None)
    
#fig.delaxes(axes[8])
fig.tight_layout()
plt.show()

In [ ]:
data_num_clean.skew()

In [ ]:
## Boxplot for numeric data

fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(15, 7), sharex = False, sharey = False)
axes = axes.ravel()  
cols = data_num_clean.columns[:]

for col, ax in zip(cols, axes):
  d = data_num_clean 
  sns.boxplot(data=d, x=col, orient = "h", palette = "Set2", width=0.3, whis=[1, 99], ax=ax) # Show the 1st and 99th percentiles
  ax.set(title=col, xlabel=None)

    
#fig.delaxes(axes[8])
fig.tight_layout()
plt.show()

### Missing and wrong values 
* Wrong row was already deleted. 
* Numeric NaN values: Replacing nan values with column means
* Non-numeric NaN values: Replacing nan values with category that has highes value count (mode)

In [ ]:
## Numeric data: Replacing nan values with column means 
for col in data_cleand.select_dtypes(include=['float64', 'int64']).columns:
    data_cleand[col] = data_cleand[col].fillna(data_cleand[col].mean())


## Non-Numeric data: Replacing nan values with category that has highes value count? (emp_title, emp_length, title)

# loop through each column and replace NaN with mode
for col in data_cleand.select_dtypes(include=['object']).columns:
        mode = data_cleand[col].mode()[0]                   # get mode of column
        data_cleand[col] = data_cleand[col].fillna(mode)    # fill NaN with mode value

data_cleand

In [ ]:
## Count NaN again
# compute number of NaN values and percentage of NaN values for each column
nan_count2 = data_cleand.isna().sum()
nan_pct2 = data_cleand.isna().sum()/n_data

# create a new dataframe to store the results
nans2 = pd.DataFrame({'nan_count': nan_count2, 'nan_pct': nan_pct2})
nans2["nan_pct_str"] = pd.Series(["{0:.2f}%".format(val * 100) for val in nans2["nan_pct"]], index = nans2.index)
nans2

In [ ]:
describe2 = data_cleand.describe()
describe2.loc['dtype'] = data_cleand.dtypes

describe2

In [ ]:
# Separating numerical and categorical data
data_num_final = data_cleand.select_dtypes(include=['float64','int64'])
data_cat_final = data_cleand.select_dtypes(include=['object'])

### Outliers 




**Outlier Detection:** https://www.freecodecamp.org/news/how-to-detect-outliers-in-machine-learning/ 

**Outlier Handling:** https://heartbeat.comet.ml/how-to-make-your-machine-learning-models-robust-to-outliers-44d404067d07 

* Try out different methods

* Delete outliers or winsorize them \
-> Reduces sensitivity of model to outliers 

In [ ]:
data_num_final.skew()

#### Winsorization
For example, a 90% winsorization sets all observations greater than the 95th percentile equal to the value at the 95th percentile and all observations less than the 5th percentile equal to the value at the 5th percentile. \
-> If no extreme outliers are present, winsorization may be unnecessary \
-> one-sided here?

https://towardsdatascience.com/detecting-and-treating-outliers-in-python-part-3-dcb54abaf7b0

In [ ]:
## Winsorization (if extreme outliers)

data_temp = data_cleand
data_cleaned_win = data_cleand.copy(deep=True)

num_col_list = data_num_final.columns.to_list()
num_col_list.remove("loan_status")


# Winsorize on right-tail
for col in num_col_list:
    data_cleaned_win[col] = winsorize(data_temp[col], limits=(0, 0.01))

data_cleaned_win.describe()


In [ ]:
data_cleaned_win.skew()

In [ ]:
## Plotting Distribution for numeric data

fig, axs = plt.subplots(nrows=3, ncols=4, figsize=(15, 7))

d = data_cleaned_win

# loan_amnt -> right/positive skewed
sns.histplot(data=d, x="loan_amnt", binwidth=1000, color="darkblue", kde=True, kde_kws={'bw_method': 0.4}, ax=axs[0,0])
# int_rate -> right/positive skewed
sns.histplot(data=d, x="int_rate", binwidth=1, color="darkblue", kde=True, kde_kws={'bw_method': 0.4}, ax=axs[0,1])
# installment -> right/positive skewed
sns.histplot(data=d, x="installment", binwidth=50, color="darkblue", kde=True, kde_kws={'bw_method': 0.4}, ax=axs[0,2])
# annual_inc -> right/positive skewed -> outlier -> re-do without
sns.histplot(data=d, x="annual_inc", binwidth=10000, color="darkblue", kde=True, kde_kws={'bw_method': 0.4}, ax=axs[0,3])
# dti -> right/positive skewed -> outlier -> re-do without
sns.histplot(data=d, x="dti", binwidth=1, color="darkblue", kde=True, kde_kws={'bw_method': 0.4}, ax=axs[1,0])
# open_acc -> right/positive skewed
sns.histplot(data=d, x="open_acc", binwidth=1, color="darkblue", kde=True, kde_kws={'bw_method': 0.4}, ax=axs[1,1])
# pub_rec -> right/positive skewed -> outlier -> re-do without
sns.histplot(data=d, x="pub_rec", binwidth=1, color="darkblue", kde=True, kde_kws={'bw_method': 0.4}, ax=axs[1,2])
# revol_bal -> right/positive skewed -> outlier -> re-do without
sns.histplot(data=d, x="revol_bal", binwidth=2000, color="darkblue", kde=True, kde_kws={'bw_method': 0.4}, ax=axs[1,3])
# revol_util -> right/positive skewed -> outlier -> re-do without
sns.histplot(data=d, x="revol_util", binwidth=2, color="darkblue", kde=True, kde_kws={'bw_method': 0.4}, ax=axs[2,0])
# total_acc -> right/positive skewed -> outlier -> re-do without
sns.histplot(data=d, x="total_acc", binwidth=1, color="darkblue", kde=True, kde_kws={'bw_method': 0.4}, ax=axs[2,1])
# mort_acc -> right/positive skewed -> outlier -> re-do without
sns.histplot(data=d, x="mort_acc", binwidth=1, color="darkblue", kde=True, kde_kws={'bw_method': 0.4}, ax=axs[2,2])
# pub_rec_bankruptcies
sns.histplot(data=d, x="pub_rec_bankruptcies", binwidth=1, color="darkblue", kde=True, kde_kws={'bw_method': 0.4}, ax=axs[2,3])

fig.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(15, 7), sharex = False, sharey = False)
axes = axes.ravel()  
cols = data_cleaned_win.select_dtypes(include=['float64', 'int64']).columns[:]

for col, ax in zip(cols, axes):
    d = data_cleaned_win.select_dtypes(include=['float64', 'int64'])
    sns.kdeplot(data=d, x=col, shade=True, ax=ax)
    ax.set(title=f'Distribution of Variable: {col}', xlabel=None)
    
#fig.delaxes(axes[8])
fig.tight_layout()
plt.show()

#### Log-Scale Transformation to detect outliers

In [ ]:
## Log-Scale Transformation (often preferred when the response variable follows exponential distribution or is right-skewed)

#### Outlier Removal

**Using the Interquartile Range (IQR)**

In [ ]:
# Get numerical variable names
num_col_list = data_num_final.columns.to_list()
num_col_list.remove("loan_status")
num_col_list

In [ ]:
data_cleaned_iqr = data_cleand

upper_limits_iqr = {}
lower_limits_iqr = {}

for col in num_col_list:

    print("\n", col, ":")

    percentile_25 = data_cleand[col].quantile(0.25)
    percentile_75 = data_cleand[col].quantile(0.75)

    print("25 percentile: ", percentile_25)
    print("75 percentile: ", percentile_75)  

    iqr = percentile_75 - percentile_25
    print("IQR: ", iqr)

    upper_limit = percentile_75 + 1.5 * iqr
    lower_limit = percentile_25 - 1.5 * iqr

    upper_limits_iqr[col] = upper_limit
    lower_limits_iqr[col] = lower_limit

    print("Upper limit: ", upper_limit)   
    print("Lower limit: ", lower_limit)

for col in num_col_list:  
    #data_test_iqr = data_test_iqr[(data_test_iqr[col] > lower_limits_iqr[col]) & (data_test_iqr[col] < upper_limits_iqr[col])]
    data_cleaned_iqr = data_cleaned_iqr[(data_cleaned_iqr[col] < upper_limits_iqr[col])]

print("\n")
print("Old Shape: ", data_cleand.shape)
print("New Shape: ", data_cleaned_iqr.shape)

diff = len(data_cleand) - len(data_cleaned_iqr)
print("Size difference: ", diff)
data_cleaned_iqr


In [ ]:
## Boxplot for numeric data

fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(15, 15), sharex = False, sharey = False)
axes = axes.ravel()  
cols = data_cleaned_iqr.select_dtypes(include=['float64', 'int64']).columns[:]

for col, ax in zip(cols, axes):
  d = data_cleaned_iqr.select_dtypes(include=['float64', 'int64'])
  #sns.boxplot(data=d, x=col, orient = "h", palette = "Set2", width=0.3, whis=[1, 99], ax=ax)
  sns.boxplot(data=d, x=col, orient = "h", palette = "Set2", width=0.3, ax=ax)
  ax.set(title=col, xlabel=None)

    
#fig.delaxes(axes[8])
fig.tight_layout()
plt.show()

**Using Percentile** \
Better for all together or single variables?


In [ ]:
data_cleaned_per = data_cleand

upper_limits_per = {}
lower_limits_per = {}

for col in num_col_list:

    print("\n", col, ":")

    upper_bound = data_cleand[col].quantile(0.99)  
    lower_bound = data_cleand[col].quantile(0.01)  

    print("99 percentile: ", upper_bound)
    print("01 percentile: ", lower_bound)  

    upper_limits_per[col] = upper_bound
    lower_limits_per[col] = lower_bound

for col in num_col_list:  
    #data_test_per = data_test_per[(data_test_per[col] > lower_limits_per[col]) & (data_test_per[col] < upper_limits_per[col])]
    data_cleaned_per = data_cleaned_per[(data_cleaned_per[col] < upper_limits_per[col])]

print("\n")
print("Old Shape: ", data_cleand.shape)
print("New Shape: ", data_cleaned_per.shape)

diff = len(data_cleand) - len(data_cleaned_per)
print("Size difference: ", diff)
data_cleaned_per

In [ ]:
## Boxplot for numeric data

fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(15, 15), sharex = False, sharey = False)
axes = axes.ravel()  
cols = data_cleaned_per.select_dtypes(include=['float64', 'int64']).columns[:]

for col, ax in zip(cols, axes):
  d = data_cleaned_per.select_dtypes(include=['float64', 'int64'])
  sns.boxplot(data=d, x=col, orient = "h", palette = "Set2", width=0.3, whis=[1, 99], ax=ax)
  ax.set(title=col, xlabel=None)

    
#fig.delaxes(axes[8])
fig.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(15, 7), sharex = False, sharey = False)
axes = axes.ravel()  
cols = data_cleaned_per.select_dtypes(include=['float64', 'int64']).columns[:]

for col, ax in zip(cols, axes):
    d = data_cleaned_per.select_dtypes(include=['float64', 'int64'])
    sns.kdeplot(data=d, x=col, shade=True, ax=ax)
    ax.set(title=f'Distribution of Variable: {col}', xlabel=None)
    
#fig.delaxes(axes[8])
fig.tight_layout()
plt.show()

In [ ]:
## Plotting Distribution for numeric data

fig, axs = plt.subplots(nrows=3, ncols=4, figsize=(15, 7))

d = data_cleaned_per

# loan_amnt -> right/positive skewed
sns.histplot(data=d, x="loan_amnt", binwidth=1000, color="darkblue", kde=True, kde_kws={'bw_method': 0.4}, ax=axs[0,0])
# int_rate -> right/positive skewed
sns.histplot(data=d, x="int_rate", binwidth=1, color="darkblue", kde=True, kde_kws={'bw_method': 0.4}, ax=axs[0,1])
# installment -> right/positive skewed
sns.histplot(data=d, x="installment", binwidth=50, color="darkblue", kde=True, kde_kws={'bw_method': 0.4}, ax=axs[0,2])
# annual_inc -> right/positive skewed -> outlier -> re-do without
sns.histplot(data=d, x="annual_inc", binwidth=10000, color="darkblue", kde=True, kde_kws={'bw_method': 0.4}, ax=axs[0,3])
# dti -> right/positive skewed -> outlier -> re-do without
sns.histplot(data=d, x="dti", binwidth=1, color="darkblue", kde=True, kde_kws={'bw_method': 0.4}, ax=axs[1,0])
# open_acc -> right/positive skewed
sns.histplot(data=d, x="open_acc", binwidth=1, color="darkblue", kde=True, kde_kws={'bw_method': 0.4}, ax=axs[1,1])
# pub_rec -> right/positive skewed -> outlier -> re-do without
sns.histplot(data=d, x="pub_rec", binwidth=1, color="darkblue", kde=True, kde_kws={'bw_method': 0.4}, ax=axs[1,2])
# revol_bal -> right/positive skewed -> outlier -> re-do without
sns.histplot(data=d, x="revol_bal", binwidth=2000, color="darkblue", kde=True, kde_kws={'bw_method': 0.4}, ax=axs[1,3])
# revol_util -> right/positive skewed -> outlier -> re-do without
sns.histplot(data=d, x="revol_util", binwidth=2, color="darkblue", kde=True, kde_kws={'bw_method': 0.4}, ax=axs[2,0])
# total_acc -> right/positive skewed -> outlier -> re-do without
sns.histplot(data=d, x="total_acc", binwidth=1, color="darkblue", kde=True, kde_kws={'bw_method': 0.4}, ax=axs[2,1])
# mort_acc -> right/positive skewed -> outlier -> re-do without
sns.histplot(data=d, x="mort_acc", binwidth=1, color="darkblue", kde=True, kde_kws={'bw_method': 0.4}, ax=axs[2,2])
# pub_rec_bankruptcies
sns.histplot(data=d, x="pub_rec_bankruptcies", binwidth=1, color="darkblue", kde=True, kde_kws={'bw_method': 0.4}, ax=axs[2,3])

fig.tight_layout()
plt.show()

### Dealing with Categorical Data 
Create Dummies for categorical Data. \
-> OneHotEncoding: https://towardsdatascience.com/an-overview-of-categorical-input-handling-for-neural-networks-c172ba552dee

Problem with OHE because of:
* earliest_cr_line 
* address 


OneHotEncoder

In [ ]:
data_cleand = data_cleaned_win      # winsorized data
# data_cleand = data_cleaned_per    # outliers removed 1%-99%-percentiles
# data_cleand = data_cleaned_iqr    # outliers removed with 25%-75%-IQR
# data_cleand = data_cleand         # data with outliers

In [ ]:
data_cleand.dtypes

In [ ]:
# Separating numerical and categorical data
data_num_final = data_cleand.select_dtypes(include=['float64','int64'])
data_cat_final = data_cleand.select_dtypes(include=['object'])

In [ ]:
# Categorical columns positions
cat_cols = data_cat_final.columns
cat_cols_list = cat_cols.to_list()
cat_col_positions = [data_cleand.columns.get_loc(col) for col in cat_cols]
display(cat_col_positions)
display(cat_cols_list)

In [ ]:
# OneHotEncoding categorical columns
ct = ColumnTransformer(
    [('ohe', pp.OneHotEncoder(sparse=False), cat_col_positions),],  # the column numbers I want to apply this to
    remainder='passthrough'  # This leaves the rest of my columns in place
)
ct_transformed = ct.fit_transform(data_cleand)
data_final = pd.DataFrame(ct_transformed, columns=ct.get_feature_names_out()) # putting array in a dataframe using column names
data_final

### Data Normalization
Standardization, Normalization or Binning approach

In [ ]:
# Should work once categorical variables one-hot-encoded

scaler = MinMaxScaler()
scaler.fit(data_final)
scaled = scaler.fit_transform(data_final)
data_final_scaled = pd.DataFrame(scaled, columns=data_final.columns)
data_final_scaled

In [ ]:
num_cols = [col for col in data_final_scaled.columns if 'remainder' in col]
cat_cols = [col for col in data_final_scaled.columns if 'remainder' not in col]

### Data Standardization

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler_std = StandardScaler()

standardized_data = scaler_std.fit_transform(data_final_scaled)

data_final_scaled_std = pd.DataFrame(standardized_data, columns=data_final.columns)
data_final_scaled_std

### Final datasets

In [ ]:
# data_final            -> not normalized, not standardized
# data_final_scaled     -> normalized
# data_final_scaled_std -> normalized, standardized

### Summary Statistics (final)

In [ ]:
data_final_scaled.describe()


### Data Visualizations (final)
Final of used factors \
(for ppt)

In [ ]:
# sns.pairplot(data_cleand, hue="loan_status")

## Correlation Analysis Variables
Find and drop highly correlated variables

In [ ]:
# Create correlation matrix
corr_matrix = data_final_scaled.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find features with correlation greater than 0.8
to_drop = [column for column in upper.columns if any(upper[column] > 0.8)] # >= 0.8 -> 5
display(to_drop)

# Drop features 
data_final_scaled = data_final_scaled.drop(to_drop, axis=1, inplace=False)
data_final_scaled

In [ ]:
# highest correlated pairs
# corr_matrix_unstacked = corr_matrix.unstack()
# sorted = corr_matrix_unstacked.sort_values(kind="quicksort")
# display(sorted)

# sorted_df = pd.DataFrame(sorted)

# display(sorted_df.loc[(sorted_df[0] >= 0.8)])
# display(sorted_df.loc[(sorted_df[0] <= -0.8)])

It calculates the correlation/strength-of-association of features in the data-set with both categorical and continuous features using: Pearson’s R for continuous-continuous cases, Correlation Ratio for categorical-continuous cases, Cramer’s V or Theil’s U for categorical-categorical cases.
associations function returns a dictionary that contains:

* ‘corr’ as key : A DataFrame of the correlation between all features.
* ‘ax’ as value: A matplotlib axe which contains the correlation heatmap.

In [ ]:
# from dython.nominal import associations

# # associations(dataset, nominal_columns='auto', numerical_columns=None, mark_columns=False, nom_nom_assoc='cramer', num_num_assoc='pearson', 
# #              bias_correction=True, nan_strategy=_REPLACE, nan_replace_value=_DEFAULT_REPLACE_VALUE, ax=None, figsize=None, annot=True, fmt='.2f', 
# #              cmap=None, sv_color='silver', cbar=True, vmax=1.0, vmin=None, plot=True, compute_only=False, clustering=False, title=None, filename=None)

# complete_correlation = associations(data_final_scaled, figsize=(10,10))

# df_complete_corr = complete_correlation['corr']
# df_complete_corr.dropna(axis=1, how='all').dropna(axis=0, how='all').style.background_gradient(cmap='coolwarm', axis=None).set_precision(2)

Correlation Matrix

In [ ]:
# only numeric variables (before normalization)
plt.figure(figsize=(10,10))
sns.heatmap(data_num_final.astype(float).corr(),linewidths=0.1,vmax=1.0, 
            square=True,  linecolor='white', annot=True)
plt.show()

In [ ]:
# numerical columns positions in final dataset
num_cols = [col for col in data_final_scaled.columns if 'remainder' in col]
num_cols

In [ ]:
# only numeric variables (after normalization -> same as before)
plt.figure(figsize=(10,10))
sns.heatmap(data_final_scaled[num_cols].astype(float).corr(),linewidths=0.1,vmax=1.0, 
            square=True,  linecolor='white', annot=True)
plt.show()

In [ ]:
plt.figure(figsize=(30,30))
sns.heatmap(data_final_scaled.astype(float).corr(),linewidths=0.1,vmax=1.0, 
            square=True,  linecolor='white', annot=True)
plt.show()

In [ ]:
data_final_scaled

## Data reality check of most important features
* loan_amnt
*